<a href="https://colab.research.google.com/github/liz435/AlternativePersona/blob/main/Persona.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install transformers datasets

In [ ]:
import pandas as pd


Load in History Data

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
BrowserHistoryJson = pd.read_json('drive/MyDrive/midterm/Takeout4/Chrome/BrowserHistory.json')
BrowserHistoryJson

,Browser History
0,{'favicon_url': 'https://www.google.com/favico...
1,{'favicon_url': 'https://ssl.gstatic.com/ui/v1...
2,{'favicon_url': 'https://ssl.gstatic.com/ui/v1...
3,{'favicon_url': 'https://duckduckgo.com/favico...
4,{'favicon_url': 'https://www.google.com/favico...
...,...
77499,{'favicon_url': 'https://du11hjcvx0uqb.cloudfr...
77500,{'favicon_url': 'https://du11hjcvx0uqb.cloudfr...
77501,{'favicon_url': 'https://du11hjcvx0uqb.cloudfr...
77502,{'favicon_url': 'https://du11hjcvx0uqb.cloudfr...


Unique Transition Types

In [16]:
transition_types = []
for i, r in BrowserHistoryJson.iterrows():
  transition_types.append((r[0]['page_transition']))
unique_transition_type  = set(transition_types)
unique_transition_type


{'AUTO_BOOKMARK',
 'AUTO_TOPLEVEL',
 'FORM_SUBMIT',
 'GENERATED',
 'LINK',
 'RELOAD',
 'TYPED'}

In [17]:
BrowserHistoryJson['Browser History'][2]


{'favicon_url': 'https://ssl.gstatic.com/ui/v1/icons/mail/rfr/gmail.ico',
 'page_transition': 'TYPED',
 'title': 'Gmail',
 'url': 'https://mail.google.com/mail/u/0/#inbox',
 'client_id': 'kyuHF2sCwM9qXzrDdrs33w==',
 'time_usec': 1662671354249329}

Seperate the search history by transition types

In [18]:
def history_category(json):
  categories = {
      "TYPED": [],
      "LINK": [],
      "RELOAD": [],
      "GENERATED": [],
      "AUTO_TOPLEVEL": [],
      "AUTO_BOOKMARK": [],
      "FORM_SUBMIT": [],
  }

  for _, row in json.iterrows():
    transition = row[0]["page_transition"]
    categories[transition].append(row[0]["title"])

  return categories

In [19]:
HistoryCategories = history_category(BrowserHistoryJson)

sentimental analysis

In [20]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [21]:
def analyzeHistory(target):
  results = {
      "TYPED": [],
      "LINK": [],
      "RELOAD": [],
      "GENERATED": [],
      "AUTO_TOPLEVEL": [],
      "AUTO_BOOKMARK": [],
      "FORM_SUBMIT": []
  }

  for i in range(len(target['LINK'])):
    results['LINK'].append(classifier(target['LINK'][i]))
  for i in range(len(target['TYPED'])):
    results['TYPED'].append(classifier(target['TYPED'][i]))
  for i in range(len(target['RELOAD'])):
    results['RELOAD'].append(classifier(target['RELOAD'][i]))
  for i in range(len(target['FORM_SUBMIT'])):
    results['FORM_SUBMIT'].append(classifier(target['FORM_SUBMIT'][i]))
  for i in range(len(target['AUTO_BOOKMARK'])):
    results['AUTO_BOOKMARK'].append(classifier(target['AUTO_BOOKMARK'][i]))
  for i in range(len(target['AUTO_TOPLEVEL'])):
    results['AUTO_TOPLEVEL'].append(classifier(target['AUTO_TOPLEVEL'][i]))

  return results
HistorySentimentOutput = analyzeHistory(HistoryCategories)

In [22]:
classifier( HistoryCategories['TYPED'][69])

[{'label': 'POSITIVE', 'score': 0.9544402956962585}]

save the sentimental analysis

In [34]:
import os
import json

In [35]:
with open('/content/drive/My Drive/Output/output.json', 'w') as f:
  json.dump(HistorySentimentOutput,f)

In [95]:
history_df = pd.DataFrame.from_dict(HistorySentimentOutput, orient = 'index').T

In [45]:
history_df.head()

,TYPED,LINK,RELOAD,GENERATED,AUTO_TOPLEVEL,AUTO_BOOKMARK,FORM_SUBMIT
0,"[{'label': 'NEGATIVE', 'score': 0.979194223880...","[{'label': 'POSITIVE', 'score': 0.956062614917...","[{'label': 'NEGATIVE', 'score': 0.755213797092...",None,"[{'label': 'NEGATIVE', 'score': 0.994368612766...","[{'label': 'POSITIVE', 'score': 0.996454238891...","[{'label': 'NEGATIVE', 'score': 0.991980552673..."
1,"[{'label': 'NEGATIVE', 'score': 0.896036982536...","[{'label': 'POSITIVE', 'score': 0.997600495815...","[{'label': 'POSITIVE', 'score': 0.934183299541...",None,"[{'label': 'POSITIVE', 'score': 0.954440295696...","[{'label': 'POSITIVE', 'score': 0.996454238891...","[{'label': 'POSITIVE', 'score': 0.751243293285..."
2,"[{'label': 'POSITIVE', 'score': 0.954440295696...","[{'label': 'POSITIVE', 'score': 0.956062614917...","[{'label': 'NEGATIVE', 'score': 0.995899021625...",None,"[{'label': 'POSITIVE', 'score': 0.954440295696...","[{'label': 'NEGATIVE', 'score': 0.994487226009...","[{'label': 'NEGATIVE', 'score': 0.926731228828..."
3,"[{'label': 'POSITIVE', 'score': 0.924298644065...","[{'label': 'POSITIVE', 'score': 0.997600495815...","[{'label': 'POSITIVE', 'score': 0.934183299541...",None,"[{'label': 'NEGATIVE', 'score': 0.574124991893...","[{'label': 'POSITIVE', 'score': 0.996454238891...","[{'label': 'NEGATIVE', 'score': 0.926731228828..."
4,"[{'label': 'POSITIVE', 'score': 0.954440295696...","[{'label': 'POSITIVE', 'score': 0.956062614917...","[{'label': 'POSITIVE', 'score': 0.934183299541...",None,"[{'label': 'POSITIVE', 'score': 0.954440295696...","[{'label': 'POSITIVE', 'score': 0.996454238891...","[{'label': 'NEGATIVE', 'score': 0.856525182723..."


In [76]:
history_df['LINK'][i][0]

{'label': 'NEGATIVE', 'score': 0.7733840942382812}

Check the score


In [65]:
sum = 0
for i in range(len(history_df['TYPED'])):
    try:
        sum += history_df['TYPED'][i][0]['score']
    except TypeError:
        pass

In [70]:
def averageSentimentScore(df):
  count = 0
  sum = 0
  for i in range(len(history_df['TYPED'])):
    try:
        count += 1
        sum += history_df['TYPED'][i][0]['score']
    except TypeError:
        pass
    return sum/count

In [71]:
averageTypedScore = averageSentimentScore(history_df)
averageTypedScore


0.9791942238807678

Check the negative score and positive score

In [83]:
def averageSentimentLINKScore(df):
  negCount = 0
  posCount = 0
  negScore = 0
  posScore = 0

  for i in range(len(history_df['LINK'])):
    try:
        if history_df['LINK'][i][0]['label'] == 'NEGATIVE':
          negCount += 1
          negScore += history_df['LINK'][i][0]['score']
        elif history_df['LINK'][i][0]['label'] == 'POSITIVE':
          posCount += 1
          posScore += history_df['LINK'][i][0]['score']
    except TypeError:
        pass

  return [negScore/negCount, negCount], [posScore/posCount, posCount]

In [87]:
averageLINKNegScore, averageLINKPosScore = averageSentimentLINKScore(history_df)
print(averageLINKNegScore, averageLINKPosScore)

[0.9037449823317507, 30359] [0.8955237227977119, 19468]


# **I experiment with the data in the above cells, but it didn't behave the way I want it to be. I might use the scores as a way to create my own randomness**

Load in Instagram messages

Chat History

In [148]:
path = {
    'park' :'/content/drive/MyDrive/midterm/davidp/message_1.json',
    'gil': '/content/drive/MyDrive/midterm/gilbertkolosieke/message_1.json',
    'mag': '/content/drive/MyDrive/midterm/magdaleneshan/message_1.json',
    'avi': '/content/drive/MyDrive/midterm/Takeout2/avi/message_1.json',
    'ch': '/content/drive/MyDrive/midterm/Takeout2/ch/message_1.json',
    'nu': '/content/drive/MyDrive/midterm/Takeout2/nu/message_1.json'
}

# **Don't know how to write this more efficently, but repeat the code 5 times**

In [149]:
with open(path['park']) as json_data:
  data = json.load(json_data)
  park_df = pd.DataFrame(data['messages'])

In [162]:
with open(path['avi']) as json_data:
  data = json.load(json_data)
  avi_df = pd.DataFrame(data['messages'])

In [156]:
with open(path['gil']) as json_data:
  data = json.load(json_data)
  gil_df = pd.DataFrame(data['messages'])

In [155]:
with open(path['ch']) as json_data:
  data = json.load(json_data)
  ch_df = pd.DataFrame(data['messages'])

In [160]:
with open(path['nu']) as json_data:
  data = json.load(json_data)
  nu_df = pd.DataFrame(data['messages'])

In [157]:
with open(path['mag']) as json_data:
  data = json.load(json_data)
  mag_df = pd.DataFrame(data['messages'])

Process Data

In [177]:
list_chat_history = [park_df, avi_df, gil_df, ch_df, nu_df, ma]

Sentiment Analysis + collect all chat

In [189]:
my_response = []

for i in list_chat_history:
  for j in range(len(i[i['sender_name'] == 'Li Zelong']['content'])):
   my_response.append(i[i['sender_name'] == 'Li Zelong']['content'].iloc[j])

664

In [216]:
classifier(my_response[1])[0]

{'label': 'POSITIVE', 'score': 0.999863862991333}

In [221]:
def response_sentiment(my_response):
  negCount = 0
  posCount = 0
  negScore = 0
  posScore = 0

  for i in my_response:
    try:
      if classifier(i)[0]['label'] == 'NEGATIVE':
        negCount += 1
        negScore += classifier(i)[0]['score']
      elif classifier(i)[0]['label'] == 'POSITIVE':
        posCount += 1
        posScore += classifier(i)[0]['score']

    except ValueError:
      pass
  return [negScore/negCount, negCount], [posScore/posCount, posCount]

In [222]:
response_score = response_sentiment(my_response)

My Interests collected by Instagram

In [174]:
with open('/content/drive/MyDrive/midterm/Takeout2/infoAbout/ads_interests.json') as json_data:
  data = json.load(json_data)
  interests_df = pd.DataFrame(data['inferred_data_ig_interest'])

In [135]:
interests_df.iloc[1][2]['Interest']['value']

'Boho-chic'

Collect in a list

In [144]:
my_interest_list = []


for _, r in interests_df.iterrows():
  my_interest_list.append(r['string_map_data']['Interest']['value'])

**Clean up the data**

In [263]:
len(my_response[6])

8

Remove non-english character due to converting error

In [286]:
def is_english(c):
  ascii_value = ord(c)
  if (ascii_value >= 65 and ascii_value <= 90):
    return True
  elif (ascii_value >= 97 and ascii_value <= 122):
    return True
  elif c ==  ' ':
    return True
  elif c == '':
    return False

def remove_non_english(lst):
  output = []
  for s in lst:
    english_str = ""
    for c in s:
      if is_english(c) == True:
        english_str += c
      else:
        pass
    output.append(english_str)
  return output

cleaned_response = remove_non_english(my_response)

# **Train the model**

In [231]:
from transformers import AutoModelForCausalLM


In [228]:
from transformers import AutoTokenizer

In [336]:
pip install accelerate -U

In [235]:
from transformers import Trainer, TrainingArguments

In [238]:
import datasets

In [278]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [ ]:
pip install tensorflow

In [299]:
with open('/content/drive/MyDrive/Output/response.txt', 'w') as f:
  for i in cleaned_response:
    f.write(i + '\n ')


In [369]:
from transformers import BertConfig, BertModel

In [317]:

from datasets import load_dataset

In [378]:
training_data = datasets.load_dataset('text', data_files= 'drive/MyDrive/Output/response.txt' )

# **Installed**

In [474]:
tokenizer.pad_token = tokenizer.eos_token

tokenized_training_data = training_data.map(lambda x: tokenizer(x['text']), remove_columns=["text"])

Map:   0%|          | 0/665 [00:00<?, ? examples/s]

In [475]:
tf_batch = tokenizer(
    cleaned_response,
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf",
    is_split_into_words=False
)

In [427]:
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tf_model = AutoModelForCausalLM.from_pretrained('distilgpt2')

In [428]:
tf_model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
tf_outputs = tf_model(tf_batch)

In [430]:
import tensorflow as tf

tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
tf_predictions

<tf.Tensor: shape=(664, 5), dtype=float32, numpy=
array([[0.31190208, 0.24365364, 0.28397802, 0.09360623, 0.06686002],
       [0.22845687, 0.2432957 , 0.21902686, 0.14615645, 0.16306415],
       [0.18278873, 0.21870628, 0.2716506 , 0.22623307, 0.10062134],
       ...,
       [0.1333641 , 0.33605626, 0.39709884, 0.11013652, 0.02334426],
       [0.22455685, 0.24213885, 0.22342712, 0.14768587, 0.16219133],
       [0.3833097 , 0.41140065, 0.16105199, 0.03199412, 0.01224353]],
      dtype=float32)>

In [431]:
tf_save_directory = "/content/drive/MyDrive/Output/Model"
tokenizer.save_pretrained(tf_save_directory)
tf_model.save_pretrained(tf_save_directory)

In [432]:
tf_model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Output/Model")

In [433]:
tokenizer = AutoTokenizer.from_pretrained(tf_save_directory)
#model = AutoModelForCausalLM.from_pretrained('distilgpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [434]:
generator = pipeline('text-generation', model=tf_model, tokenizer=tokenizer)

In [436]:
import random

In [452]:
word = random.choice(my_interest_list)
sentence = f'I love {word}'
sentence

'I love British Shorthair'

In [453]:
generator(sentence)[0]['generated_text']

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'I love British Shorthair."\n"We know he is coming, and what do we do. Why do we see him?"'

In [460]:
def generateSentence(s):
  word = random.choice(s)
  sentence = f'I love {word}'
  print(generator(sentence)[0]['generated_text'])


In [483]:
generateSentence(my_interest_list)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


I love Child development...I hope you guys like this one too.
And then we find out about the latest and greatest child development trends!
